In [ ]:
import torch 
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import numpy as np 
import time 
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split
import math
import string

In [2]:
train_data_1=torch.load('train_data_50_mut_1_c.pt')
test_data_1=torch.load('test_data_50_mut_1_c.pt')
train_labels_1=torch.load('train_labels_50_mut_1_c.pt')
test_labels_1=torch.load('test_labels_50_mut_1_c.pt')

In [ ]:
num_train_words=len(train_data_1)
num_test_words=len(test_data_1)

In [75]:
num_train_words=48042 # 1000 words 50 misspilled single mistake 
num_test_words=5338 

In [4]:
device= torch.device("cuda")

In [5]:
class Recurrent_Layer(nn.Module):
    
    def __init__(self, vocab_size, hidden_size):
        super(Recurrent_Layer, self).__init__()
        

        self.layer1 = nn.Embedding (vocab_size, hidden_size)
        self.lstm=nn.LSTM( hidden_size, hidden_size, bidirectional=True)
        self.dropout=nn.Dropout(0.2)
        self.fc=nn.Linear( 2* hidden_size, vocab_size)

    def forward(self, x, h_init,c_init):

        g_seq          =self.layer1(x)
        h_seq, (h_final, c_final) =self.lstm(g_seq, (h_init,c_init))
        h_seq=self.dropout(h_seq)
        score_seq      =self.fc(h_seq)
        
        return score_seq,  h_final , c_final

In [76]:
hidden_size=256
vocab_size =26
# word_length= 19 #  according to the data 
batch_of_words=1

lstm_net=Recurrent_Layer(vocab_size,hidden_size).cuda()

lstm_net = lstm_net.to(device)
print(lstm_net)
criterion = nn.CrossEntropyLoss()
my_lr=0.05
accu_plt=[]
loss_plt=[]
test_loss_plt=[]
accuracy_test_plt=[]

Recurrent_Layer(
  (layer1): Embedding(26, 256)
  (lstm): LSTM(256, 256, bidirectional=True)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=26, bias=True)
)


## Utilities

In [66]:
def get_error( scores , labels ):

    bs=scores.size(0)
    predicted_labels = scores.argmax(dim=1)
    indicator = (predicted_labels == labels)
    num_matches=indicator.sum()
    corr=num_matches/len(predicted_labels)
    return corr.item()

def save_model_parameters():
# save models 
    torch.save({
                'epoch': epoch,
                'lstm_model':lstm_net.state_dict(),
                'lstm_optimizer': lstm_optimizer.state_dict(),
                'train_loss': loss_plt, 'train_accuracy': accu_plt, 
                'test_loss': test_loss_plt,
                'test_accuracy': accuracy_test_plt, 'confusion matrix_parameters':confusion_mtx_parameters  }, 'lstm_single_char_50_mutation_bid_drop_02.pt')
#                 'incorrect_to_correct':confusion_mtx_parameters[0],  'correct_to_correct':confusion_mtx_parameters[2],
#                 'correct_to_incorrect':confusion_mtx_parameters[1],'regenerate':confusion_mtx_parameters[3]
  
           
               
     
        #print(in_word, out_word)
        
def confusion_parameters(scores,target_tensor,inpute_tensor,conf_counter):

    
    if torch.all(torch.eq(target_tensor, scores.argmax(dim=1)))==1 and torch.all(torch.eq(inpute_tensor,target_tensor))==0: ### making incorrect->correct
        conf_counter[0] +=1

    if torch.all(torch.eq(inpute_tensor, scores.argmax(dim=1)))==0 and torch.all(torch.eq(inpute_tensor,target_tensor))==1: ### making correct->incorrect

        conf_counter[1] +=1    


    if torch.all(torch.eq(inpute_tensor, scores.argmax(dim=1)))==1 and torch.all(torch.eq(inpute_tensor,target_tensor))==1: ### making correct->correct

        conf_counter[2] +=1                                                               

    if torch.all(torch.eq(inpute_tensor, scores.argmax(dim=1)))==1 and torch.all(torch.eq(inpute_tensor,target_tensor))==0: ### making inccorrect->correct or regenerate the input 

        conf_counter[3] +=1  
    return conf_counter




## Evaluation

In [55]:
def eval_on_test_set():
    # to deactivate dropout regularization during testing
    lstm_net.eval()
    running_loss=0
    num_batches=0    
    num_matches=0 
    acc_plt=[]
    loss_plt=[]
    
    # counts for computing correct and incorrect proportion
    count=torch.zeros(4)
    cnt_list=torch.zeros(4)
    cnf_mtx_count=torch.zeros(4)
    
    for i in range(0,num_test_words):
        
        h = torch.zeros(2,batch_of_words,  hidden_size).cuda()
        c = torch.zeros(2,batch_of_words,  hidden_size).cuda()
        h=h.to(device)
        c=c.to(device)
        word_length= len(test_data_1 [i])
        
        inpute_tensor=  test_data_1[i].cuda()
        target_tensor =  test_labels_1[i].cuda()
        
        # sending to GPU
        inpute_tensor=inpute_tensor.to(device)
        target_tensor=target_tensor.to(device)
        # forward pass
        scores_char, h, c= lstm_net(inpute_tensor.view(word_length,1), h, c)
        # reshape before calculating the loss for easier slicing of mini batch of words
        scores_char = scores_char.view(word_length*batch_of_words,vocab_size)
        target_tensor = target_tensor.contiguous()
        target_tensor = target_tensor.view(word_length*batch_of_words)
        # calculating the loss of batch of character sthat constrcut a words
        loss_char= criterion(scores_char, target_tensor)# do we need to add the loss for every 
        
            #================================================#
        # accumalate the loss
        running_loss+= loss_char.item()
        num_batches+=1 
        # computing accuracy
        num_matches+= get_error(scores_char, target_tensor)
        #=======================================================#
        # compute confusion paramters 
        cnf_mtx_count+=confusion_parameters(scores_char,target_tensor,inpute_tensor,count)
        #============================================# 
        

    ###====Different Metrics for evaluation=============###
    
    # counter for computing confusion matrix
#     cnt_list[0]+=cnf_mtx_count[0].item()
#     cnt_list[1]+=cnf_mtx_count[1].item()
#     cnt_list[2]+=cnf_mtx_count[2].item()
#     cnt_list[3]+=cnf_mtx_count[3].item()
  
    # accuracy and loss 
    accuracy= (num_matches/num_test_words)*100
    acc_plt.append(accuracy)
    total_loss = running_loss/num_batches 
    loss_plt.append(total_loss)
    
    # printing results 
    print('Test==: loss = ',(total_loss),'\t accuracy=', accuracy,'%' )
    return acc_plt, loss_plt, cnf_mtx_count

## Training 

In [56]:
eval_on_test_set()

Test==: loss =  0.6082124173641205 	 accuracy= 20.0 %


([20.0], [0.6082124173641205], tensor([9., 1., 6., 0.]))

In [77]:
start=time.time()
# shuff_index=torch.LongTensor(485224).random_(0,485224)
# train_data_2=train_data_1[:,shuff_index]
# train_labels_2=train_labels_1[:,shuff_index]

accu_plt=[]
loss_plt=[]
test_loss_plt=[]
accuracy_test_plt=[]
confusion_mtx_parameters=[]

for epoch in range(1,1000):
    # to activate dropout during training 
    lstm_net.train()
    # divide the learning rate by 3 except after the first epoch
    if epoch % 10==0:
        my_lr = my_lr / 1.1

    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    lstm_optimizer=torch.optim.SGD( lstm_net.parameters() , lr=my_lr )
    # set the initial h and c to be the zero vector

        
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
    num_matches=0
    
    # loop across batch of words
    for i in range(0,num_train_words):
      # initilize the hidden state every word as the words ara independent 
    
        h = torch.zeros(2,batch_of_words,  hidden_size).cuda()
        c = torch.zeros(2,batch_of_words,  hidden_size).cuda()
        h=h.to(device)
        c=c.to(device)
    
        word_length= len(train_data_1 [i])
    # Set the gradients to zeros
        lstm_optimizer.zero_grad()

        minibatch_words  =  train_data_1[i ]
        minibatch_labels =  train_labels_1[ i]
        # sending to GPU
        minibatch_words=minibatch_words.to(device)
        minibatch_labels=minibatch_labels.to(device)


        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h=h.detach()
        c=c.detach()
        h=h.requires_grad_()
        c=c.requires_grad_()

        # forward pass
        scores_char, h, c = lstm_net(minibatch_words.view(word_length,1), h, c)

        # reshape before calculating the loss for easier slicing of mini batch of words
        scores_char = scores_char.view(  word_length*batch_of_words , vocab_size)
        minibatch_labels = minibatch_labels.contiguous()
        minibatch_labels = minibatch_labels.view(word_length*batch_of_words )
        # calculating the loss of batch of character sthat constrcut a words
        loss_char= criterion(scores_char, minibatch_labels)

        #===============================================

        # summation of both lossses to 
        combined_loss=loss_char


        # backward pass to compute dL/dR, dL/dV and dL/dW
        combined_loss.backward()

        # update the wights 
        lstm_optimizer.step()


        # update the running loss  
        running_loss += combined_loss.detach().item()
        num_batches += 1
        num_matches += get_error(scores_char, minibatch_labels)
        
        # end of iteration 
        
    # compute for full training set
    total_loss = running_loss/num_batches
    loss_plt.append(total_loss)
    accuracy= (num_matches/num_train_words)*100
    accu_plt.append(accuracy)
    # Compute the time 
    elapsed = time.time()-start
    
    print('')
    print('Train:::', 'epoch=',epoch,'\t lr=', my_lr, '\t (loss)=',(total_loss),'\t (accuracy)=' , (accuracy),'%','\t time=', elapsed)
    
    # evaluate on test set to monitor the loss 
    acc_tst_plt, loss_tst_plt, confusion_param=eval_on_test_set()
    
    # saving test parameters  
    test_loss_plt.append(loss_tst_plt)
    accuracy_test_plt.append(acc_tst_plt)
    confusion_mtx_parameters.append(confusion_param)
    #### Saving model parameters every epoch 
    save_model_parameters() 
        


Train::: epoch= 1 	 lr= 0.05 	 (loss)= 0.18853896301434886 	 (accuracy)= 64.87032180175679 % 	 time= 164.76657223701477
Test==: loss =  1.3834352731935906 	 accuracy= 2.660172349194455 %

Train::: epoch= 2 	 lr= 0.05 	 (loss)= 0.1304543384860051 	 (accuracy)= 74.77831897090046 % 	 time= 341.44678688049316
Test==: loss =  1.2325140768421163 	 accuracy= 4.664668415136755 %

Train::: epoch= 3 	 lr= 0.05 	 (loss)= 0.11625305223941712 	 (accuracy)= 77.24282919112443 % 	 time= 516.7183020114899
Test==: loss =  1.1888244432584683 	 accuracy= 4.777070063694268 %

Train::: epoch= 4 	 lr= 0.05 	 (loss)= 0.10650655914574439 	 (accuracy)= 78.71862120644437 % 	 time= 692.4947218894958
Test==: loss =  1.1607471884599676 	 accuracy= 5.545147995503934 %

Train::: epoch= 5 	 lr= 0.05 	 (loss)= 0.0987752673410938 	 (accuracy)= 80.17359810166104 % 	 time= 867.3575296401978
Test==: loss =  1.0624884485962 	 accuracy= 7.680779318096666 %

Train::: epoch= 6 	 lr= 0.05 	 (loss)= 0.09143425864279349 	 (accur


Train::: epoch= 42 	 lr= 0.03415067276825353 	 (loss)= 0.015157261144410662 	 (accuracy)= 96.37608759002539 % 	 time= 7355.519688844681
Test==: loss =  0.46278255459782386 	 accuracy= 49.11952041963282 %

Train::: epoch= 43 	 lr= 0.03415067276825353 	 (loss)= 0.014527000389397794 	 (accuracy)= 96.40731027018026 % 	 time= 7529.79181599617
Test==: loss =  0.4189266533096916 	 accuracy= 51.04908205320344 %

Train::: epoch= 44 	 lr= 0.03415067276825353 	 (loss)= 0.014524065967371597 	 (accuracy)= 96.44477748636609 % 	 time= 7703.372896671295
Test==: loss =  0.40271216578480595 	 accuracy= 52.43536905207943 %

Train::: epoch= 45 	 lr= 0.03415067276825353 	 (loss)= 0.013754167504134275 	 (accuracy)= 96.63835810332625 % 	 time= 7878.5569949150085
Test==: loss =  0.37879656203729334 	 accuracy= 54.75833645560135 %

Train::: epoch= 46 	 lr= 0.03415067276825353 	 (loss)= 0.013491409904441158 	 (accuracy)= 96.83402023229674 % 	 time= 8052.830947160721
Test==: loss =  0.37004305951247035 	 accura


Train::: epoch= 82 	 lr= 0.02332536901048666 	 (loss)= 0.008027475286808663 	 (accuracy)= 98.16202489488364 % 	 time= 15696.700811624527
Test==: loss =  0.13611755429975733 	 accuracy= 81.50992881228925 %

Train::: epoch= 83 	 lr= 0.02332536901048666 	 (loss)= 0.007905288109577158 	 (accuracy)= 98.23487781524499 % 	 time= 16019.828091144562
Test==: loss =  0.1342184443195652 	 accuracy= 81.62233046084675 %

Train::: epoch= 84 	 lr= 0.02332536901048666 	 (loss)= 0.007761153387806281 	 (accuracy)= 98.29940468756504 % 	 time= 16343.59430384636
Test==: loss =  0.13410610233532294 	 accuracy= 81.11652304233795 %

Train::: epoch= 85 	 lr= 0.02332536901048666 	 (loss)= 0.007589368049227663 	 (accuracy)= 98.38266516797802 % 	 time= 16667.892414808273
Test==: loss =  0.1330988277852082 	 accuracy= 81.97826901461221 %

Train::: epoch= 86 	 lr= 0.02332536901048666 	 (loss)= 0.00768719894099721 	 (accuracy)= 98.24944839931726 % 	 time= 16993.262807369232
Test==: loss =  0.12739110775783657 	 accu


Train::: epoch= 122 	 lr= 0.01593154088551783 	 (loss)= 0.006396772765662182 	 (accuracy)= 98.70946255359894 % 	 time= 28724.955037117004
Test==: loss =  0.07882265573962266 	 accuracy= 88.40389659048333 %

Train::: epoch= 123 	 lr= 0.01593154088551783 	 (loss)= 0.006532235842934829 	 (accuracy)= 98.65950626535115 % 	 time= 29049.8575463295
Test==: loss =  0.07888742699900195 	 accuracy= 88.51629823904084 %

Train::: epoch= 124 	 lr= 0.01593154088551783 	 (loss)= 0.006323787176003763 	 (accuracy)= 98.6990549935473 % 	 time= 29375.160282611847
Test==: loss =  0.07689629879738616 	 accuracy= 88.81603596852754 %

Train::: epoch= 125 	 lr= 0.01593154088551783 	 (loss)= 0.006188036723690964 	 (accuracy)= 98.75109279380541 % 	 time= 29702.017313957214
Test==: loss =  0.07757745238801836 	 accuracy= 88.40389659048333 %

Train::: epoch= 126 	 lr= 0.01593154088551783 	 (loss)= 0.006022611497881458 	 (accuracy)= 98.80729361808417 % 	 time= 30028.611914634705
Test==: loss =  0.07692535582635802 

KeyboardInterrupt: 

In [33]:
print(confusion_mtx_parameters)

[[[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0], [1.0], [0.0], []], [[1.0], [1.0]

In [ ]:
checkpoint = torch.load('lstm_single_char_50_mutation_bid_drop.pt')

In [10]:
torch.save(lstm_net, 'this_morning_w_do.pth')

/home/andriashfahani/miniconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Recurrent_Layer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
print(checkpoint['test_loss'])

In [ ]:
 torch.save(lstm_net,'single_mistake_model_lstm_drop_out_1.pth')

In [68]:
checkpoint = torch.load('lstm_single_char_50_mutation_bid_drop_02.pt')

In [74]:
print(checkpoint[])
#  'incorrect_to_correct':confusion_mtx_parameters[0],  'correct_to_correct':confusion_mtx_parameters[2],
# #                 'correct_to_incorrect':confusion_mtx_parameters[1],'regenerate':confusion_mtx_parameters[3]
  
           

[0.008481162786483764, 0.009953457117080688, 0.008462512493133545, 0.008568865060806275, 0.008872705698013305, 0.009778851270675659, 0.009399718046188355, 0.009138405323028564, 0.008280891180038451, 0.00938764214515686, 0.009248465299606323, 0.010755127668380738, 0.009009695053100586, 0.008971762657165528, 0.01076977252960205, 0.01007135510444641, 0.010158282518386842, 0.008264505863189697, 0.008282959461212158, 0.01121429204940796, 0.008798831701278686, 0.009055852890014648, 0.010330718755722047, 0.007766813039779663, 0.009448951482772827, 0.009692203998565675, 0.00912320613861084, 0.009875631332397461, 0.008551460504531861, 0.01096159815788269, 0.009085208177566528, 0.009600162506103516, 0.00832710862159729, 0.010008686780929565, 0.009891873598098755, 0.010521680116653442, 0.010294747352600098, 0.009758317470550537, 0.010009801387786866, 0.008804881572723388, 0.008309823274612427, 0.009526288509368897, 0.008488410711288452, 0.008953988552093506, 0.009343469142913818, 0.00920448899269